# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [51]:
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution, easy_datetime_distribution
import sys
import os
sys.path.append(os.path.realpath('../'))
from medusa.edge_extractor import *
from medusa.matrix_modifier import MatrixModifier
from medusa.medusa_lca import *
from medusa.timeline_builder import TimelineBuilder
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [52]:
from tests.databases import *
db_abc_loopA_with_biosphere()

100%|██████████| 1/1 [00:00<00:00, 33288.13it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 45343.83it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 47662.55it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 4/4 [00:00<00:00, 97541.95it/s]

Vacuuming database 


In [53]:
demand = {('foreground', 'A'): 1}
gwp = ('GWP', 'example')

In [54]:
SKIPPABLE = [node.id for node in bd.Database('background_2008')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

In [55]:
import warnings
warnings.filterwarnings("ignore")

database_date_dict = {
            'background_2008': datetime.strptime("2008", "%Y"),
            'background_2024': datetime.strptime("2024", "%Y"),
            'foreground': "dynamic", # flag databases that should be temporally distributed with "dynamic"
        }

In [56]:
# mlca = MedusaLCA(demand, gwp, filter_function, database_date_dict, max_calc=10)
mlca = MedusaLCA(demand, gwp, filter_function, database_date_dict, max_calc=1000)

Starting graph traversal
Calculation count: 42


In [57]:
mlca.build_timeline()

All databases in database_date_dict exist as brightway project databases


,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,1995,10,1995-01-01,2,C,2000,15,2000-01-01,8,B,13.0,{'background_2008': 1}
1,1996,11,1996-01-01,2,C,2001,16,2001-01-01,8,B,13.0,{'background_2008': 1}
2,1997,12,1997-01-01,2,C,2002,17,2002-01-01,8,B,13.0,{'background_2008': 1}
3,1998,13,1998-01-01,2,C,2003,18,2003-01-01,8,B,13.0,{'background_2008': 1}
4,1999,14,1999-01-01,2,C,2004,19,2004-01-01,8,B,13.0,{'background_2008': 1}
...,...,...,...,...,...,...,...,...,...,...,...,...
79,2035,70,2035-01-01,8,B,2020,46,2020-01-01,9,A,0.4,{'background_2024': 1}
80,2036,71,2036-01-01,8,B,2021,50,2021-01-01,9,A,0.4,{'background_2024': 1}
81,2037,72,2037-01-01,8,B,2022,54,2022-01-01,9,A,0.4,{'background_2024': 1}
82,2038,73,2038-01-01,8,B,2023,58,2023-01-01,9,A,0.4,{'background_2024': 1}


In [58]:
mlca.build_datapackage()

In [59]:
mlca.lci()

In [60]:
mlca.lcia()

In [61]:
print('New MEDUSA LCA Score:', mlca.score)
print('Old static LCA Score:', mlca.static_lca.score)

New MEDUSA LCA Score: 240.30568505509464
Old static LCA Score: 240.5555559538397


In [62]:
time_mapping_dict_reversed = {value: key for key, value in mlca.time_mapping_dict.items()}

df = pd.DataFrame(mlca.technosphere_matrix.toarray())

df.rename(mlca.dicts.activity.reversed, inplace=True, axis=0)
df.rename(mlca.dicts.activity.reversed, inplace=True, axis=1)
df.rename(time_mapping_dict_reversed, inplace=True, axis=0)
df.rename(time_mapping_dict_reversed, inplace=True, axis=1)
df

,"((background_2024, C), 2024)","((background_2024, electricity_wind), 2024)","((background_2008, C), 2008)","((background_2008, electricity_wind), 2008)","((foreground, E), dynamic)","((foreground, D), dynamic)","((foreground, B), dynamic)","((foreground, A), dynamic)","((exploded, C), 1995)","((exploded, C), 1996)",...,"((exploded, C), 2030)","((exploded, C), 2031)","((exploded, C), 2032)","((exploded, C), 2033)","((exploded, C), 2034)","((exploded, B), 2035)","((exploded, B), 2036)","((exploded, B), 2037)","((exploded, B), 2038)","((exploded, B), 2039)"
"((background_2024, C), 2024)",1.0,0.0,0.0,0.0,-11.0,0.0,-13.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
"((background_2024, electricity_wind), 2024)",-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"((background_2008, C), 2008)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"((background_2008, electricity_wind), 2008)",0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"((foreground, E), dynamic)",0.0,0.0,0.0,0.0,1.0,-2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"((exploded, B), 2035)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"((exploded, B), 2036)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"((exploded, B), 2037)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"((exploded, B), 2038)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
